# Gurobi

Gurobi es un solver de optimización comercial, incluye:

- Linear programming (LP)
- Quadratic programming (QP)
- Quadratically constrained programming (QCP)
- Mixed integer linear programming (MILP)
- Mixed-integer quadratic programming (MIQP)
- Mixed-integer quadratically constrained programming (MIQCP).
- Non-Convex Quadratic Optimization (NCQO)

Hay varias formas de programar un modelo de optimización usando Gurobi en Julia, ejemplos en:
- https://github.com/JuliaOpt/Gurobi.jl

Benchmark:
- https://www.gurobi.com/pdfs/benchmarks.pdf

Se recomienda usar Gurobi con JuMP, <a hred="http://www.juliaopt.org/JuMP.jl/stable/">JuMP</a> es un lenguage de optimización matemática que soporta un gran abanico de solvers.

In [1]:
#using Pkg
#Pkg.add(PackageSpec(url="https://github.com/JuliaOpt/JuMP.jl/", rev="master"))
#Pkg.add("Gurobi")
#Pkg.add("GLPK") #solver
#Pkg.add("Test") #librería para testear
#Pkg.add("DataFrames")
#Pkg.add("CSV")
using JuMP, Gurobi, GLPK, Test, DataFrames, CSV

# 1. Simple LP

\begin{equation}
\begin{matrix}
\underset{x,y}{\min} & x+y \\
\textrm{s.t.} & 50x+24y & \leq & 2400 \\
& 30x+33y & \leq & 2100\\
& x & \geq & 45\\
& y & \geq & 5\\
\end{matrix}
\end{equation}

Presolve: operaciones que realiza un solver para transformar un problema de optimización en uno equivalente más fácil de resolver.

In [1]:
#Creamos el objeto modelo

#Le indicamos a JuMP que el solver a utilizar es Gurobi

#Seteamos opciones del solver, en este caso desactivamos el Presolve
 

#declaración de variables de decisión

#restricciones, el nombre de la restricción es opcional

#función objetivo

#resolver


In [2]:
#imprimir modelo


In [3]:
#acceder a una variable del modelo desde el modelo


In [4]:
#acceder a una variable del modelo desde la referencia


In [5]:
#acceder a una restricción del modelo desde el modelo


In [ ]:
#acceder a una restricción del modelo desde la referencia


In [11]:
#preguntar por qué el solver paró


In [12]:
#verifical el estado de la solución primal


In [13]:
#verifical el estado de la solución dual 


In [6]:
#Imprimir valor óptimo, 

#Imprimir solución primal

#Imprimir solución dual

#Imprimir valores de las restricciones

In [7]:
#veremos que pasa si obligamos al que problema primal sea no acotado

#resolver


# 2. Diet Problem

Supongamos que hay $n$ diferentes comidas y $m$ diferentes nutrientes, y tenemos la siguiente tabla de contenido nutricial por cada unidad de comida:

|            |  food 1  | ... |  food n  |
|:----------:|:--------:|:---:|:--------:|
| nutrient 1 | $a_{11}$ | ... | $a_{1n}$ |
|   &#8942;  |  &#8942; |     |  &#8942; |
| nutrient m | $a_{m1}$ | ... | $a_{mn}$ |

Sea $A$ una matrix $mxn$ con entradas $a_{ij}$, notar que la columna $A_{j}$ representa el contenido nutricional de la comida $j$, sea $b$ un vector con los requirimientos nutricionales de una dieta ideal, sea $c$ el vector de costos de cada unidad de comida. El problema anterior se puede representar como LP en forma estándar:

\begin{equation}
\begin{matrix}
\underset{x}{\min} & c^{´}x \\
\textrm{s.t.} & Ax = b\\
& x \geq 0\\
\end{matrix}
\end{equation}

Es decir se busca la dieta que satisfaga los requirimientos nutricionales de una dieta ideal a costo mínimo.

In [17]:
#Creamos una instancia

#nutrientes
nutrients = ["calorías", "proteína", "grasa", "carbohidratos"]

#comidas
foods = ["hamburguesa", "completo italiano", "papas fritas", "pizza", "fideos con salsa",
         "porotos con mazamorra", "lentejas", "pastel de choclo", "platano", "leche",
         "yogurt", "batido"]

#requerimientos dieta ideal
b = JuMP.Containers.DenseAxisArray(
    [
    2000, #kcal
    120, #proteínas [gr]
    40, #grasas [gr]
    250 #carbohidratos [gr] 
    ], nutrients)

#vector de costos en pesos chilenos por unidad de alimento
c = JuMP.Containers.DenseAxisArray(
    [
    1400, #hamburguesa
    1450, #completo italiano
    550, #papas fritas
    750, #pizza
    2500, #fideos con salsa
    2000, #porotos con mazamorra
    1500, #lentejas
    3000, #pastel de choclo
    300, #platano
    250, #leche
    400, #yogurt
    600 #batido
    ], foods)

#matriz A transpuesta, (comida, nutriente [gr])
At = JuMP.Containers.DenseAxisArray(
    [
    482 21 26 41; #hamburguesa 
    440 12 24 44; #completo italiano 1450
    502 4 22 72; #papas fritas 550
    612.5 32.5 40.1 30.4; #pizza 750
    329 20 13 33; #fideos con salsa 
    336 14 12 43; #porotos con mazamorra
    358.8 27 1.2 60; #lentejas
    567 21 27 60; #pastel de choclo
    204.2 1.9 0.6 47.8; #platano
    71.6 9 0.4 8; #leche
    92.4 14 0.8 7.3; #yogurt
    230 23 0 8; #batido
    ], foods, nutrients);

In [21]:
#guardamos la instancia en un diccionario
diet_data = Dict("b"=>b, "c"=>c, "At"=>At)

Dict{String,JuMP.Containers.DenseAxisArray} with 3 entries:
  "c"  => 1-dimensional DenseAxisArray{Int64,1,...} with index sets:…
  "b"  => 1-dimensional DenseAxisArray{Int64,1,...} with index sets:…
  "At" => 2-dimensional DenseAxisArray{Float64,2,...} with index sets:…

In [8]:
function diet_model(solver, diet_data, verbose)
   
    #obtener parámetros de diet_data y guardar en variables
    
    #testear ingreso correcto de parámetros
        #testear dimensiones
        #testear positividad de b,c y no negatividad de At
    
    #lista de comidas y nutrientes
    
    #crear modelo y setear solver
    
    #cantidad a consumir por comida
    
    #restricciones por nutriente, la suma de los nutrientes por alimento debe ser igual al requerimiento ideal
    
    #Minimizar costo de la dieta ideal
    
    #Resolver
    
    
    #Status de término del solver
   
    #Si verbose is true
  
        #Si se alcanzó el óptimo mostrar solución óptima, sino mostrar mensaje que no se alcanzó

    #Retornar modelo
end

diet_model (generic function with 1 method)

In [ ]:
idiet_model = diet_model(GLPK.Optimizer, diet_data, true);

In [ ]:
idiet_model = diet_model(Gurobi.Optimizer, diet_data, true);

# 3. Piecewise linear function and Data fitting

## Piecewise linear function

Una función lineal por partes se puede representar como:

$f(x) = \underset{i=1,\ldots,m}{max}(c_{i}^{´}+d_{i})$

Donde $x, c_{i} \in \rm I\!R^{n}, d_{i} \in \rm I\!R,  \forall  i=1,\ldots,m$. Notar que esta función es convexa, de hecho se puede demostrar que el máximo de funciones convexas es convexo. 

Supongamos que nos enfrentamos al problema de buscar el $x$ que minimize $f(x)$:

$\underset{x}{min} \underset{i=1,\ldots,m}{max}(c_{i}^{´}+d_{i})$

El problema anterior se puede escribir como un LP:

\begin{equation}
\begin{matrix}
\underset{x, z}{\min} & z \\
\textrm{s.t.} & z & \geq & c^{´}_{i}x+d_{i} & i=1,\ldots,m\\
\end{matrix}
\end{equation}

## Data fitting
Supongamos que contamos con $m$ observaciones de la forma ($x_{i}, y_{i}$), $i=1,\ldots,m$, donde $x_{i} \in \rm I\!R^{n}$ y $y_{i}\in\rm I\!R$, deseamos construir un modelo que prediga el valor de $y$ dado $x$. En tal caso usualmente se utiliza un modelo lineal de la forma $y=\beta^{´}x$, donde $\beta$ es un vector de parámetros a estimar. Sea $|y_{i}-\beta^{´}x_{i}|$ el residuo o error de predicción en la observación $i$, generalmente se busca determinar el vector de parámetros que minimiza el error cuadrático medio, pero en este caso buscaremos el vector de parámetros que minimice el residuo más grande:

$\underset{\beta}{min} \underset{i=1,\ldots,m}{max}|y_{i}-\beta^{'}x_{i}|$

Notar que la función modulo es una función lineal por partes, ya que $|x|=max\{x,-x\}$. El problema anterior se puede reescribir como un LP:

\begin{equation}
\begin{matrix}
\underset{\beta, z}{\min} & z \\
\textrm{s.t.} & y_{i}-\beta^{'}x_{i} & \leq z & i=1,\ldots,m\\
& -y_{i}+\beta^{'}x_{i} & \leq z & i=1,\ldots, m
\end{matrix}
\end{equation}

Para el ejemplo se utilizará la data `01.Housing.csv`, la base de datos posee precios de casas de algunas localidades de USA junto a un par de covariables que se correlacionan con este.

In [25]:
#ajustar variable de entorno para ver todas las columnas del dataframe
ENV["COLUMNS"]=150

150

In [223]:
#importar .csv y crear un dataframe que lo almacena
df = CSV.read("01.Housing.csv");
df.bias = [1 for i in 1:size(df)[1]]
#printear primeras 5 filas del dataframe
first(df, 5)

,Avg Area Income,Avg Area House Age,Avg Area Number of Rooms,Avg Area Number of Bedrooms,Area Population,Price,bias
,Float64,Float64,Float64,Float64,Float64,Float64,Int64
1,79545.5,5.68286,7.00919,4.09,23086.8,1.05903e6,1
2,79248.6,6.0029,6.73082,3.09,40173.1,1.50589e6,1
3,61287.1,5.86589,8.51273,5.13,36882.2,1.05899e6,1
4,63345.2,7.18824,5.58673,3.26,34310.2,1.26062e6,1
5,59982.2,5.04055,7.83939,4.23,26354.1,6.30943e5,1


In [224]:
columns = String.(names(df))
setdiff!(columns, ["Price"])

6-element Array{String,1}:
 "Avg Area Income"            
 "Avg Area House Age"         
 "Avg Area Number of Rooms"   
 "Avg Area Number of Bedrooms"
 "Area Population"            
 "bias"                       

In [225]:
features = [Symbol(col) for col in columns]
target = Symbol("Price")
#vector de características
x = convert(Matrix, df[:,features]);
#variable objetivo a predecir
y = df[:,target];
#almacenamos la data en un diccionario
data = Dict("x"=>x, "y"=>y)

Dict{String,Array{Float64,N} where N} with 2 entries:
  "x" => [79545.5 5.68286 … 23086.8 1.0; 79248.6 6.0029 … 40173.1 1.0; … ; 68001.3 5.53439 … 42625.6 1.0; 65510.6 5.99231 … 46501.3 1.0]
  "y" => [1.05903e6, 1.50589e6, 1.05899e6, 1.26062e6, 6.30943e5, 1.06814e6, 1.50206e6, 1.57394e6, 7.9887e5, 1.54515e6  …  4.79501e5, 1.26372e6, 1.568…

In [10]:
function fit_model(solver, data, verbose)

    #obtener datos
    x, y = data["x"], data["y"]
    m, n = size(x) 
    
    #crear modelo
   
    
    #variables
    
    
    #restricciones
    
    #función objetivo
    
    #resolver
   
    
    #Status de término del solver
    
    #Si verbose is true
        #Si se alcanzó el óptimo mostrar solución óptima, sino mostrar mensaje que no se alcanzó
    
    #Retornar modelo
end

fit_model (generic function with 1 method)

In [ ]:
#Resolver con Gurobi
ifit_model = fit_model(Gurobi.Optimizer, data, true);

In [ ]:
#Resolver con GLPK
ifit_model = fit_model(GLPK.Optimizer, data, true);

In [ ]:
#Solución óptima
β = 

In [ ]:
#Inferir valor óptima a través de la función objetivo
residuals  = 

In [ ]:
#Valor óptimo solver ≈ Valor óptimo inferido
